# July 12, 13, 16, 17

In [2]:


import torch
import torch.nn.functional as F
import time

dist_matrix = torch.rand(1000, 120)
print(dist_matrix.shape)
num_nearest_neighbors = 5

start = time.time() 
ind = torch.topk(dist_matrix, num_nearest_neighbors, largest=False).indices
end = time.time() 

print(f"Time taken to run the function: {end - start} seconds")
print(ind.shape)


torch.Size([1000, 120])
Time taken to run the function: 0.0036056041717529297 seconds
torch.Size([1000, 5])


In [3]:
import torch
import numpy as np

B, C, N = 32, 12, 40
n_samples = 5
k_neig = 5

ex = torch.rand(B, C, N)
print("Ex shape: ", ex.shape)

M = ex[0]/torch.norm(ex[0], dim=0)  # Pick first item in patch and normalize the rows
print("M shape: ", M.shape)

rand_idx = np.random.choice(M.shape[1], n_samples, replace=False) 
print(rand_idx)

M_prime = M[:, rand_idx]
print("M prime shape: ", M_prime.shape)

sim = M.T@M_prime
print("Sim shape: ", sim.shape)


# Make similarities between samples equal to -inf, so they are not considered when using topk
#sim[rand_idx, np.arange(n_samples)]  =  -np.inf  
# print(sim[:rand_idx[0]])
# print(sim[:rand_idx[0]].shape)


# Only get k-1 neigbors (we'll add the selves later as the first nearest neigbor) 
_, indxs = torch.topk(sim, k_neig - 1, largest=True) 
b = [rand_idx[x] for x in indxs]
neig1 = M[:, b]
print("Neig shape: ", neig1.shape)
print("M.unsqueeze(2) shape: ", M.unsqueeze(2).shape)

# Add the selves as the first nearest neigbor
neig2 = torch.concat([M.unsqueeze(2), neig1], dim=2)
print(neig2.shape)


Ex shape:  torch.Size([32, 12, 40])
M shape:  torch.Size([12, 40])
[ 9 36 34 38 10]
M prime shape:  torch.Size([12, 5])
Sim shape:  torch.Size([40, 5])
Neig shape:  torch.Size([12, 40, 4])
M.unsqueeze(2) shape:  torch.Size([12, 40, 1])
torch.Size([12, 40, 5])


/var/folders/s7/md5b66vn4l39df_j063nqlp00000gn/T/ipykernel_47693/2407026106.py:33: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:278.)
  neig1 = M[:, b]


In [4]:
print(rand_idx)
c = rand_idx[0]
print(c)
torch.concat([torch.arange(40).unsqueeze(1), sim], dim=1)

[ 9 36 34 38 10]
9


tensor([[ 0.0000,  0.7503,  0.8438,  0.7944,  0.7955,  0.6967],
        [ 1.0000,  0.6766,  0.6655,  0.7897,  0.7534,  0.8060],
        [ 2.0000,  0.5189,  0.7829,  0.7517,  0.8064,  0.6501],
        [ 3.0000,  0.7753,  0.7779,  0.7532,  0.6207,  0.5696],
        [ 4.0000,  0.6310,  0.7638,  0.8394,  0.5881,  0.7308],
        [ 5.0000,  0.7185,  0.8168,  0.7329,  0.7316,  0.5488],
        [ 6.0000,  0.5998,  0.7095,  0.7235,  0.7819,  0.6019],
        [ 7.0000,  0.8311,  0.8239,  0.6703,  0.7740,  0.4814],
        [ 8.0000,  0.8263,  0.8148,  0.7159,  0.8161,  0.7571],
        [ 9.0000,  1.0000,  0.7943,  0.6116,  0.7064,  0.5001],
        [10.0000,  0.5001,  0.5486,  0.8512,  0.6431,  1.0000],
        [11.0000,  0.6797,  0.7357,  0.8085,  0.8417,  0.7498],
        [12.0000,  0.7170,  0.8001,  0.7876,  0.7893,  0.7586],
        [13.0000,  0.6181,  0.7710,  0.5917,  0.6989,  0.5950],
        [14.0000,  0.7383,  0.7222,  0.7238,  0.5803,  0.5989],
        [15.0000,  0.7492,  0.7962,  0.5

In [5]:
M[:, c]

tensor([0.5267, 0.0521, 0.3913, 0.2414, 0.0314, 0.5558, 0.1473, 0.2186, 0.1122,
        0.3036, 0.0699, 0.1393])

In [6]:
neig1[:, c, :].T

tensor([[0.5267, 0.0521, 0.3913, 0.2414, 0.0314, 0.5558, 0.1473, 0.2186, 0.1122,
         0.3036, 0.0699, 0.1393],
        [0.1500, 0.0591, 0.2692, 0.2927, 0.1862, 0.4599, 0.3333, 0.4179, 0.4505,
         0.1580, 0.0495, 0.2315],
        [0.0526, 0.0525, 0.2882, 0.1481, 0.1473, 0.3530, 0.1390, 0.3723, 0.4461,
         0.4086, 0.4474, 0.1384],
        [0.3753, 0.2861, 0.0714, 0.2012, 0.1917, 0.1101, 0.3585, 0.4051, 0.0747,
         0.0452, 0.4327, 0.4420]])

In [7]:
neig2[:, c, :].T

tensor([[0.5267, 0.0521, 0.3913, 0.2414, 0.0314, 0.5558, 0.1473, 0.2186, 0.1122,
         0.3036, 0.0699, 0.1393],
        [0.5267, 0.0521, 0.3913, 0.2414, 0.0314, 0.5558, 0.1473, 0.2186, 0.1122,
         0.3036, 0.0699, 0.1393],
        [0.1500, 0.0591, 0.2692, 0.2927, 0.1862, 0.4599, 0.3333, 0.4179, 0.4505,
         0.1580, 0.0495, 0.2315],
        [0.0526, 0.0525, 0.2882, 0.1481, 0.1473, 0.3530, 0.1390, 0.3723, 0.4461,
         0.4086, 0.4474, 0.1384],
        [0.3753, 0.2861, 0.0714, 0.2012, 0.1917, 0.1101, 0.3585, 0.4051, 0.0747,
         0.0452, 0.4327, 0.4420]])

# I. Distance/Similarity Matrix check

### Distance matrix sanity check for all samples

In [8]:
def calculate_distance_matrix(matrix): 
   '''Calculating the distance matrix of the input matrix'''
   norm_squared = torch.sum(matrix ** 2, dim=1, keepdim=True)
   dot_product = torch.bmm(matrix.transpose(2, 1), matrix)
   dist_matrix = norm_squared + norm_squared.transpose(2, 1) - 2 * dot_product
   return torch.sqrt(dist_matrix) # May need to remove torch.sqrt - do not need that computation


### Distance Matrix sanity check for N Sample 

In [9]:
def calculate_distance_matrix_N(m1, m2):
   '''Calculate the distance matrix between two input matrices'''
   # to make it simple -> normalize the rows, normalize across c (channels), 
   norm_squared_1 = torch.sum(m1 ** 2, dim=1, keepdim=True)
   norm_squared_2 = torch.sum(m2 ** 2, dim=1, keepdim=True).transpose(2, 1)
   dot_product = torch.bmm(m1.transpose(2, 1), m2)

   norm_squared_1 = norm_squared_1.permute(0, 2, 1)  
   norm_squared_2 = norm_squared_2.permute(0, 2, 1)  

   dist_matrix = norm_squared_1 + norm_squared_2 - 2 * dot_product
   return torch.sqrt(dist_matrix)


In [10]:
def dist_matrix(m1, m2): 
   # Calculate the distance matrix using for-loops 
   dist_matrix = torch.zeros(m1.shape[0], m1.shape[2], m2.shape[2])
   
   for i in range(m1.shape[0]): 
      for j in range(m1.shape[2]): 
            for k in range(m2.shape[2]): 
               dist_matrix[i, j, k] = torch.norm(m1[i, :, j] -  m2[i, :, k])
   return dist_matrix

In [11]:
m1 = torch.rand(32, 12, 40)
m2 = torch.rand(32, 12, 5)

a = calculate_distance_matrix_N(m1, m2)
b = dist_matrix(m1, m2)
print(a.shape)
print(b.shape)

torch.Size([32, 40, 5])
torch.Size([32, 40, 5])


In [12]:
print(a[0, 0, :])
print(b[0, 0, :]) 

print(torch.allclose(a, b))

print(a[0, :3, :])
print(b[0, :3, :])

tensor([1.1190, 1.5861, 1.1530, 1.4294, 1.6749])
tensor([1.1190, 1.5861, 1.1530, 1.4294, 1.6749])
True
tensor([[1.1190, 1.5861, 1.1530, 1.4294, 1.6749],
        [1.0520, 1.4970, 1.1459, 1.2078, 1.7169],
        [1.1073, 1.5688, 1.2173, 1.2805, 1.3759]])
tensor([[1.1190, 1.5861, 1.1530, 1.4294, 1.6749],
        [1.0520, 1.4970, 1.1459, 1.2078, 1.7169],
        [1.1073, 1.5688, 1.2173, 1.2805, 1.3759]])


### Similarity Matrix sanity check for N Sample

In [13]:
def sim_a(matrix): 
   '''Calculate the similarity matrix of the input matrix'''
   normalized_matrix = F.normalize(matrix, p=2, dim=1) # p=2 (L2 Norm - Euclidean Distance), dim=1 (across the channels)
   dot_product = torch.bmm(normalized_matrix.transpose(2, 1), normalized_matrix)
   similarity_matrix = dot_product 
   return similarity_matrix


In [14]:
def sim_b(m):
    # Normalize the input matrix
    normalized_matrix = F.normalize(m, p=2, dim=1)
    
    # Initialize the similarity matrix with zeros
    similarity_matrix = torch.zeros(m.size(0), m.size(2), m.size(2))
    
    # Calculate the similarity (dot product) between each pair of vectors
    for i in range(m.shape[0]): 
      for j in range(m.shape[2]): 
            for k in range(m.shape[2]): 
               similarity_matrix[i, j, k] = torch.dot(normalized_matrix[i, :, j],  normalized_matrix[i, :, k])
    
    return similarity_matrix


In [15]:
m = torch.rand(32, 12, 40)
sim1 = sim_a(m)
sim2 = sim_b(m)

print(sim1.shape)
print(sim2.shape)


print(sim1[0, 0, :3])
print(sim2[0, 0, :3]) 

print(torch.allclose(sim1, sim2))



torch.Size([32, 40, 40])
torch.Size([32, 40, 40])
tensor([1.0000, 0.6249, 0.6762])
tensor([1.0000, 0.6249, 0.6762])
True


In [16]:
## For N Samples

def sim_c(m1, m2): 
   normalized_matrix_1 = F.normalize(m1, p=2, dim=1) # p=2 (L2 Norm - Euclidean Distance), dim=1 (across the channels)
   normalized_matrix_2 = F.normalize(m2, p=2, dim=1)
   
   dot_product = torch.bmm(normalized_matrix_1.transpose(2, 1), normalized_matrix_2)
            
   similarity_matrix = dot_product 
   return similarity_matrix

def sim_d(m1, m2): 
   
   norm1 = F.normalize(m1, p=2, dim=1)
   norm2 = F.normalize(m2, p=2, dim=1)
       
    # Initialize the similarity matrix with zeros
   similarity_matrix = torch.zeros(m.size(0), m1.size(2), m2.size(2))

   # Calculate the similarity (dot product) between each pair of vectors
   for i in range(m1.shape[0]): 
      for j in range(m1.shape[2]): 
            for k in range(m2.shape[2]): 
               similarity_matrix[i, j, k] = torch.dot(norm1[i, :, j],  norm2[i, :, k])

   return similarity_matrix

In [17]:
m1 = torch.rand(32, 12, 40)
m2 = torch.rand(32, 12, 5)

sim3 = sim_c(m1, m2)
sim4 = sim_d(m1, m2)
print("shape")
print(sim3.shape)
print(sim4.shape)
print()

print("sample")
print(sim3[0, 0:3, :3])
print(sim4[0, 0:3, :3])
print()

print(torch.allclose(sim3, sim4))

shape
torch.Size([32, 40, 5])
torch.Size([32, 40, 5])

sample
tensor([[0.7096, 0.8937, 0.7985],
        [0.8496, 0.7131, 0.7314],
        [0.7478, 0.7187, 0.7948]])
tensor([[0.7096, 0.8937, 0.7985],
        [0.8496, 0.7131, 0.7314],
        [0.7478, 0.7187, 0.7948]])

True


# II. Prime comparisons

### Sanity check for topk for distance prime

In [18]:
def prime(matrix, dist_matrix, num_nearest_neighbors): 
   '''Returns the convolution matrix of the NNT object using for-loops and vectorization'''
   ### Note: For-Loop Implementation and Vectorization Implementation are the same
   
   ##For-Loop Implementation
   
   stack_list = [] 
   
   for i in range(matrix.shape[0]): 
      
       concat_list = [] 
       for j in range(matrix.shape[2]): 
           # Get the indices of the nearest neighbors
           indices = torch.topk(dist_matrix[i, j, :], num_nearest_neighbors, largest=False).indices
            
           # Get the nearest neighbors
           nearest_neighbors = matrix[i, :, indices]
            
           # Concatenate the nearest neighbors
           concat_list.append(nearest_neighbors)
      
       # Concatenate the tensor list to create the convolution matrix 
       concat = torch.cat(concat_list, dim=1)
       stack_list.append(concat)
   prime = torch.stack(stack_list, dim= 0)
   
   
   
   return prime

def prime_vmap_2d(matrix, magnitude_matrix, num_nearest_neighbors, largest=False): 
   # Vectorization / Vmap Implementation for Nearest Neighbor Tensor 2D
   batched_process = torch.vmap(process_batch, in_dims=(0, 0, None), out_dims=0)
   prime = batched_process(matrix, magnitude_matrix, num_nearest_neighbors, flatten=True, largest=largest)
   return prime 

def process_batch(matrix, magnitude_matrix, num_nearest_neighbors, flatten=True, largest=False):
      '''Process the batch of matrices''' 
      ind = torch.topk(magnitude_matrix, num_nearest_neighbors, largest=largest).indices
      neigh = matrix[:, ind]
      if flatten: 
         reshape = torch.flatten(neigh, start_dim=1)
         return reshape
      else: 
         return neigh
      


In [19]:
m = torch.rand(32, 12, 40)
dist = calculate_distance_matrix(m)

p1 = prime(m, dist, 5)
p2 = prime_vmap_2d(m, dist, 5)

In [20]:
print(p1[0, 0, :3])
print(p2[0, 0, :3])

print(torch.allclose(p1, p2))

tensor([0.4591, 0.4045, 0.3830])
tensor([0.4591, 0.4045, 0.3830])
True


### Sanity check for similarity prime

In [21]:
def sim_prime(matrix, sim_mat, num_nearest_neighbors): 
   '''Returns the convolution matrix of the NNT object using for-loops and vectorization'''
   ### Note: For-Loop Implementation and Vectorization Implementation are the same
   
   ##For-Loop Implementation
   
   stack_list = [] 
   
   for i in range(matrix.shape[0]): 
      
       concat_list = [] 
       for j in range(matrix.shape[2]): 
           # Get the indices of the nearest neighbors
           indices = torch.topk(sim_mat[i, j, :], num_nearest_neighbors, largest=True).indices
            
           # Get the nearest neighbors
           nearest_neighbors = matrix[i, :, indices]
            
           # Concatenate the nearest neighbors
           concat_list.append(nearest_neighbors)
      
       # Concatenate the tensor list to create the convolution matrix 
       concat = torch.cat(concat_list, dim=1)
       stack_list.append(concat)
   prime = torch.stack(stack_list, dim= 0)
   
   
   return prime

def sim_prime_vmap_2d(matrix, magnitude_matrix, num_nearest_neighbors, largest=True): 
   # Vectorization / Vmap Implementation for Nearest Neighbor Tensor 2D
   batched_process = torch.vmap(process_batch, in_dims=(0, 0, None), out_dims=0)
   prime = batched_process(matrix, magnitude_matrix, num_nearest_neighbors, flatten=True, largest=largest)
   return prime 

def sim_process_batch(matrix, magnitude_matrix, num_nearest_neighbors, flatten=True, largest=False):
      '''Process the batch of matrices''' 
      ind = torch.topk(magnitude_matrix, num_nearest_neighbors, largest=largest).indices
      neigh = matrix[:, ind]
      if flatten: 
         reshape = torch.flatten(neigh, start_dim=1)
         return reshape
      else: 
         return neigh

In [22]:
m = torch.rand(32, 12, 40)
sim = sim_a(m)

p1 = sim_prime(m, sim, 5)
p2 = sim_prime_vmap_2d(m, sim, 5)

In [138]:
print(p1[0, 0:3, :3])
print(p2[0, 0:3, :3])

print(torch.allclose(p1, p2))

print(p1.shape)

tensor([[0.0216, 0.4228, 0.2916],
        [0.4888, 0.0307, 0.2975],
        [0.9265, 0.3255, 0.5084]])
tensor([[0.0216, 0.4228, 0.2916],
        [0.4888, 0.0307, 0.2975],
        [0.9265, 0.3255, 0.5084]])
False
torch.Size([32, 12, 120])


### Distance prime for N sample

In [27]:
# Og shape = [32, 12, 40] 
# prime1 shape = [32, 40, 5]
# prime shape = [32, 12, 200]

indices_list1 = [] 
indices_list2 = [] 

def prime_brute_N(matrix, magnitude_matrix, num_nearest_neighbors, rand_idx, largest = False):         
   stack_list = [] 

   for i in range(matrix.shape[0]): 
      
      concat_list = []
      
      for j in range(matrix.shape[2]): 
            
         if j in rand_idx: 
            indices = torch.topk(magnitude_matrix[i, j, :], num_nearest_neighbors, largest=largest).indices
            indices_list = [rand_idx[i] for i in indices]
            nearest_neighbors = matrix[i, :, indices_list]
         else: 
            indices = torch.topk(magnitude_matrix[i, j, :], num_nearest_neighbors - 1, largest=largest).indices
            indices_list = [j] + [rand_idx[i] for i in indices]
            nearest_neighbors = matrix[i, :, indices_list]
            
         indices_list1.append(indices_list)
            
         concat_list.append(nearest_neighbors)
      
      stacked_neighbors = torch.cat(concat_list, dim=1)
      stack_list.append(stacked_neighbors)
   

   prime = torch.stack(stack_list, dim= 0)
   return prime

def prime_brute_N_v2(matrix, magnitude_matrix, num_nearest_neighbors, rand_idx,largest=False): 
   stack_list = [] 
   
   for i in range(matrix.shape[0]): 
      concat_list = [] 
      
      for j in range(matrix.shape[2]): 
         indices = torch.topk(magnitude_matrix[i, j, :], num_nearest_neighbors - 1, largest=largest).indices
         indices_list = [j] + [rand_idx[i] for i in indices] 
         nearest_neighbors = matrix[i, :, indices_list]
         concat_list.append(nearest_neighbors)
   
         indices_list2.append(indices_list)
         
      stacked_neighbors = torch.cat(concat_list, dim=1)
      stack_list.append(stacked_neighbors)
   
   prime = torch.stack(stack_list, dim=0)
   return prime

rand_idx = [3, 8, 14, 28, 39]
nnn = 3

m1 = torch.rand(32, 12, 40)
m2 = m1[:, :, rand_idx]
dist = calculate_distance_matrix_N(m1, m2)

p1 = prime_brute_N(m1, dist, nnn, rand_idx, False)

dist[:, rand_idx, np.arange(len(rand_idx))] = np.inf

p2 = prime_brute_N_v2(m1, dist, nnn, rand_idx, False) 

print(torch.equal(p1, p2))

print(p1[0, 0, :20])
print(p2[0, 0, :20])


# differing_indices = [i for i, (a, b) in enumerate(zip(indices_list1, indices_list2)) if a != b]

# # Print the indices
# print(differing_indices)
# print(indices_list1[3])
# print(indices_list2[3])


False
tensor([0.0216, 0.4228, 0.2916, 0.7075, 0.8836, 0.2916, 0.0033, 0.2916, 0.9318,
        0.8836, 0.9318, 0.9885, 0.2550, 0.9318, 0.4228, 0.8846, 0.8836, 0.9885,
        0.7070, 0.9318])
tensor([0.0216, 0.4228, 0.2916, 0.7075, 0.8836, 0.2916, 0.0033, 0.2916, 0.9318,
        0.8836, 0.9318, 0.9885, 0.2550, 0.9318, 0.4228, 0.8846, 0.8836, 0.9885,
        0.7070, 0.9318])


In [103]:
differing_indices = [i for i, (a, b) in enumerate(zip(indices_list1, indices_list2)) if a != b]

# Print the indices
print(differing_indices)
print(indices_list1[39])
print(indices_list2[39])

print(indices_list1[54])
print(indices_list2[54])

print(indices_list2[0:3])

[14, 39, 54, 68, 119, 159, 203, 248, 283, 288, 323, 414, 479, 488, 508, 519, 534, 748, 799, 808, 814, 854, 879, 999, 1148, 1268]
[28, 3, 14]
[39, 28, 3]
[39, 8, 3]
[14, 39, 8]
[[0, 14, 8], [1, 3, 8], [2, 8, 28]]


In [136]:
import torch

def prime_vmap_2d_N(matrix, magnitude_matrix, num_nearest_neighbors, rand_idx, largest=False):
    index = torch.arange(0, matrix.shape[2])
    
    batched_process = torch.vmap(process_batch_N, in_dims=(0, 0, None, None), out_dims=0)
    prime = batched_process(matrix, magnitude_matrix, num_nearest_neighbors, rand_idx, flatten=True, largest=largest)
    return prime

def process_batch_N(matrix, magnitude_matrix, num_nearest_neighbors, rand_idx, flatten=True, largest=False):
    topk_ind = torch.topk(magnitude_matrix, num_nearest_neighbors - 1, largest=largest).indices
    mapped_tensor = rand_idx[topk_ind]
    index_tensor = torch.arange(0, matrix.shape[1]).unsqueeze(1) # shape [40, 1]
    final_tensor = torch.cat([index_tensor, mapped_tensor], dim=1)
         
    neigh = matrix[:, final_tensor]
    if flatten:
        reshape = torch.flatten(neigh, start_dim=1)
        return reshape
    else:
        return neigh

In [137]:
rand_idx = torch.tensor([3, 8, 14, 28, 39])
rand_idx = [3, 8, 14, 28, 39]


print(m1.shape) 
print(dist.shape)
p3 = prime_vmap_2d_N(m1, dist, nnn, torch.tensor(rand_idx), False)
print(p3.shape)

torch.Size([32, 12, 40])
torch.Size([32, 40, 5])
torch.Size([32, 12, 120])


In [128]:
print(torch.allclose(p2, p3))
print(torch.allclose(p1, p3))
print(torch.allclose(p1, p2))

True
False
False


In [104]:
import torch

# Example tensor of size [40, 2]
original_tensor = torch.randint(0, 5, (40, 2))

# Mapping list: the index corresponds to the number in the original tensor
mapping_list = [3, 8, 14, 28, 39]

# Convert the mapping list to a tensor
mapping_tensor = torch.tensor(mapping_list)

# Use the original tensor to index the mapping tensor
mapped_tensor = mapping_tensor[original_tensor]

# Create a tensor that adds numbers from 0 to 39
index_tensor = torch.arange(40).unsqueeze(1)  # shape [40, 1]

# Concatenate the index tensor with the mapped tensor
final_tensor = torch.cat((index_tensor, mapped_tensor), dim=1)

print(original_tensor[:3, :])
print(mapped_tensor[:3, :])
print(final_tensor[:3, :])

tensor([[4, 4],
        [4, 4],
        [4, 0]])
tensor([[39, 39],
        [39, 39],
        [39,  3]])
tensor([[ 0, 39, 39],
        [ 1, 39, 39],
        [ 2, 39,  3]])


In [25]:
rand_idx = [3, 8, 14, 28, 39]
nnn = 3

m1 = torch.rand(32, 12, 40)
m2 = m1[:, :, rand_idx]
dist = calculate_distance_matrix_N(m1, m2)

dist[:, rand_idx, np.arange(len(rand_idx))] = np.inf
print(dist[0, 0:10, :])
print(dist.shape)

ind = torch.topk(dist, nnn-1, largest=False).indices
print(ind.shape)


tensor([[1.0155, 1.5307, 1.4502, 1.3405, 1.4851],
        [1.0813, 1.6507, 1.6002, 1.3544, 1.1218],
        [1.3391, 1.4990, 1.1786, 2.0250, 1.5569],
        [   inf, 1.5706, 1.3242, 1.5635, 1.4475],
        [1.5554, 1.6324, 1.6491, 1.7839, 1.2585],
        [0.9596, 1.5664, 1.1294, 1.6351, 1.5507],
        [1.0040, 1.5696, 1.3818, 1.7687, 1.4577],
        [1.2212, 1.4889, 1.6367, 1.2570, 1.0519],
        [1.5706,    inf, 0.8682, 1.3536, 1.3824],
        [1.4758, 1.0443, 1.2268, 1.7363, 1.4955]])
torch.Size([32, 40, 5])
torch.Size([32, 40, 2])
